In [ ]:
from grid.interface import *

In [ ]:
from grid import vast

In [ ]:
vast.offers('cuda_max_good >= 11.0 & rentable & gpu_name == "RTX 2080 Ti"', default=True)

In [ ]:
vast.stats().set_index('date').resample('15min').mean().plot()

In [ ]:
jittens.manage.tails('output/pavlov/*/logs.0.txt', count=500, lineglob='*Approx*')

In [ ]:
jittens.jobs.jobs()['2021-02-19 14-31-32 pesky-trays']